# Bab 15: Processing Sequences Using RNNs, CNNs, and Transformers

### 1. Pendahuluan

Bab 15 membahas Jaringan Saraf Tiruan (JST) yang dirancang khusus untuk memproses data sekuensial atau urutan, seperti teks, suara, video, atau deret waktu. **Recurrent Neural Networks (RNNs)** adalah jenis JST yang memiliki "memori" internal yang memungkinkan mereka mempertahankan informasi dari langkah waktu sebelumnya, menjadikannya sangat cocok untuk tugas-tugas yang melibatkan dependensi temporal.

Bab ini akan membahas:
* Arsitektur RNN dasar dan tantangan pelatihannya.
* Varian RNN yang lebih canggih seperti **Long Short-Term Memory (LSTM)** dan **Gated Recurrent Unit (GRU)** yang mengatasi masalah memori jangka pendek.
* Penggunaan RNN untuk tugas peramalan deret waktu (*time series forecasting*).
* Arsitektur **WaveNet** yang menggunakan lapisan konvolusional 1D untuk memproses urutan secara efisien.

---

### 2. Recurrent Neural Networks (RNNs)

Berbeda dengan jaringan *feedforward* standar, RNN memiliki koneksi berulang (*recurrent connections*) yang memungkinkan informasi untuk bertahan. Sebuah sel RNN menerima input dari langkah waktu saat ini dan juga output dari langkah waktu sebelumnya. *State* tersembunyi ($h_t$) ini bertindak sebagai bentuk memori.

#### a. Input dan Output Sequences
RNN dapat menangani berbagai jenis tugas berdasarkan input dan outputnya:
* **Sequence-to-sequence:** Menerjemahkan kalimat.
* **Sequence-to-vector:** Analisis sentimen (input kalimat, output kelas sentimen).
* **Vector-to-sequence:** Membuat *caption* untuk gambar.

#### b. Masalah Pelatihan RNN
RNN seringkali sulit dilatih karena masalah *vanishing/exploding gradients* (seperti pada DNN) yang terjadi sepanjang dimensi waktu. Ketika urutan data sangat panjang, gradien bisa menghilang, membuat model kesulitan belajar dependensi jangka panjang.

---

### 3. Peramalan Deret Waktu (Time Series Forecasting)

RNN sangat cocok untuk peramalan deret waktu, seperti memprediksi harga saham atau cuaca.

#### a. Mempersiapkan Data
Untuk melatih RNN pada data deret waktu, kita perlu mengubahnya menjadi jendela data (*windowed dataset*). Setiap *instance* terdiri dari jendela input (misalnya, 20 langkah waktu sebelumnya) dan jendela target (misalnya, 1 langkah waktu berikutnya atau beberapa langkah ke depan).

#### b. Baseline Metrics
Sebelum membangun model yang kompleks, penting untuk memiliki metrik dasar. Pendekatan naif seperti memprediksi bahwa nilai berikutnya akan sama dengan nilai saat ini seringkali menjadi *baseline* yang baik.

#### c. Implementasi Simple RNN dengan Keras

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Fungsi untuk menghasilkan data deret waktu sintetis
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    time = time[np.newaxis, :, np.newaxis]  # shape (1, n_steps, 1)
    offsets1 = offsets1[:, np.newaxis, :]   # shape (batch_size, 1, 1)
    offsets2 = offsets2[:, np.newaxis, :]   # shape (batch_size, 1, 1)
    freq1 = freq1[:, np.newaxis, :]         # shape (batch_size, 1, 1)
    freq2 = freq2[:, np.newaxis, :]         # shape (batch_size, 1, 1)
    wave1 = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))
    wave2 = 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))
    noise = 0.1 * (np.random.rand(batch_size, n_steps, 1) - 0.5)
    series = wave1 + wave2 + noise
    return series.astype(np.float32)

# Membuat dataset
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

# Model RNN sederhana
model_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

optimizer = keras.optimizers.Adam(learning_rate=0.005)
model_rnn.compile(loss="mse", optimizer=optimizer)
# history = model_rnn.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

c:\Users\Khalif Prawira\AppData\Local\Programs\Orange\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### d. Deep RNNs
Menumpuk beberapa lapisan RNN dapat menghasilkan performa yang lebih baik. Penting untuk mengatur `return_sequences=True` untuk semua lapisan RNN kecuali lapisan terakhir.

In [4]:
# Model Deep RNN
model_deep_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1) # Lapisan terakhir hanya mengembalikan output akhir
])

---

### 4. Menangani Dependensi Jangka Panjang
Masalah utama RNN sederhana adalah mereka memiliki memori jangka pendek karena masalah vanishing gradients. Untuk mengatasi ini, digunakan sel yang lebih canggih.

### a. Sel LSTM (Long Short-Term Memory)
Sel LSTM adalah varian RNN yang sangat sukses. Ia memiliki mekanisme gerbang (gate mechanism) yang secara eksplisit mengontrol informasi mana yang harus diingat dan dilupakan.

* Forget Gate: Mengontrol informasi mana dari long-term state yang harus dibuang.
* Input Gate: Mengontrol informasi mana dari input saat ini yang harus disimpan di long-term state.
* Output Gate: Mengontrol informasi mana dari long-term state yang harus dibaca dan dikeluarkan sebagai output pada langkah waktu saat ini.
Mekanisme ini memungkinkan LSTM untuk belajar dan mengingat dependensi dalam urutan yang sangat panjang.

In [5]:
# Model dengan lapisan LSTM
model_lstm = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20),
    keras.layers.Dense(1)
])

### b. Sel GRU (Gated Recurrent Unit)
Sel GRU adalah versi yang lebih sederhana dari LSTM. Ia menggabungkan cell state dan hidden state serta forget gate dan input gate menjadi satu update gate. Arsitekturnya yang lebih sederhana membuatnya sedikit lebih cepat untuk dilatih.

---

### 5. Arsitektur WaveNet untuk Urutan Data
Arsitektur WaveNet, yang awalnya diusulkan oleh DeepMind untuk audio generatif, juga sangat baik untuk data deret waktu. Ia menggunakan tumpukan lapisan konvolusional 1D (Conv1D) dengan dilated convolutions.

* **Causal Convolutions**: Lapisan konvolusional yang memastikan output pada langkah waktu `t` hanya bergantung pada input dari langkah waktu `t` dan sebelumnya, bukan dari masa depan.
* **Dilated Convolutions**: Dengan melompati input pada setiap langkah (dilation rate), satu lapisan dapat memiliki receptive field yang sangat besar, memungkinkannya untuk menangkap pola jangka panjang dengan lebih efisien daripada RNN.

In [ ]:
# Model WaveNet untuk peramalan multi-langkah
# (Memprediksi 10 langkah ke depan)
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train, Y_valid = Y[:7000], Y[7000:9000]

model_wavenet = keras.models.Sequential()
model_wavenet.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2: # Tumpukan blok dilatasi
    model_wavenet.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal",
                                 activation="relu", dilation_rate=rate))
model_wavenet.add(keras.layers.Conv1D(filters=10, kernel_size=1)) # Lapisan output

# Compile dan latih model
# model_wavenet.compile(loss="mse", optimizer="adam", ...)
# history = model_wavenet.fit(X_train, Y_train[..., -1], ...)

Arsitektur ini dapat jauh lebih cepat untuk dilatih daripada RNN, terutama untuk urutan data yang sangat panjang.